In [1]:
from pymongo import MongoClient
import pandas as pd

In [30]:
client = MongoClient("mongodb+srv://gatikamanaswinivarma:<KpwPPfu8BjJ8UB4x>@cluster0.xxajs.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")

db = client["hotel_guests"]

In [32]:
collection = db["dining_info"]

In [34]:
df_from_mongo = pd.DataFrame(list(collection.find()))

ServerSelectionTimeoutError: SSL handshake failed: cluster0-shard-00-00.xxajs.mongodb.net:27017: [('SSL routines', '', 'tlsv1 alert internal error')] (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: cluster0-shard-00-01.xxajs.mongodb.net:27017: [('SSL routines', '', 'tlsv1 alert internal error')] (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: cluster0-shard-00-02.xxajs.mongodb.net:27017: [('SSL routines', '', 'tlsv1 alert internal error')] (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67b1bc9a9b85c830482d0514, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('cluster0-shard-00-00.xxajs.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect("SSL handshake failed: cluster0-shard-00-00.xxajs.mongodb.net:27017: [('SSL routines', '', 'tlsv1 alert internal er

In [25]:
!pip install "pymongo[srv]"


In [27]:
client = MongoClient("mongodb+srv://gatikamanaswinivarma:<KpwPPfu8BjJ8UB4x>@cluster0.xxajs.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")

db = client["hotel_guests"]

In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss


df = pd.read_excel(r"C:\Users\VINNI\OneDrive\Desktop\infosys internship\dining_info.xlsx")

df['order_time'] = pd.to_datetime(df['order_time'])

# Splitting Data
features_df = df[df['order_time'] < '2024-01-01']
train_df = df[(df['order_time'] >= '2024-01-01') & (df['order_time'] <= '2024-10-01')]
test_df = df[df['order_time'] > '2024-10-01']

# Feature Engineering
customer_features = features_df.groupby("customer_id").agg(
    total_orders_per_customer=("transaction_id", "count"),
    avg_spend_per_customer=("price_for_1", "mean"),
    total_qty_per_customer=("Qty", "sum")
).reset_index()

cuisine_features = features_df.groupby("Preferred Cusine").agg(
    avg_price_per_cuisine=("price_for_1", "mean"),
    total_orders_per_cuisine=("transaction_id", "count")
).reset_index()

# Merge features into train/test datasets
train_df = train_df.merge(customer_features, on="customer_id", how="left")
train_df = train_df.merge(cuisine_features, on="Preferred Cusine", how="left")
test_df = test_df.merge(customer_features, on="customer_id", how="left")
test_df = test_df.merge(cuisine_features, on="Preferred Cusine", how="left")

# Drop unnecessary columns (including datetime columns)
cols_to_drop = [
    "transaction_id", "customer_id", "order_time", 
    "Qty", "price_for_1", "check_in_date", "check_out_date"
]

train_df.drop(cols_to_drop, axis=1, inplace=True)
test_df.drop(cols_to_drop, axis=1, inplace=True)

# Encoding Categorical Data
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cuisine = encoder.fit_transform(train_df[["Preferred Cusine"]])
train_df = pd.concat([train_df, pd.DataFrame(encoded_cuisine)], axis=1).drop(columns=["Preferred Cusine"])

test_cuisine = encoder.transform(test_df[["Preferred Cusine"]])
test_df = pd.concat([test_df, pd.DataFrame(test_cuisine)], axis=1).drop(columns=["Preferred Cusine"])

# Encode target variable
y_train = LabelEncoder().fit_transform(train_df.pop("dish"))
y_test = LabelEncoder().fit_transform(test_df.pop("dish"))

# Train XGBoost Classifier
xgb_model = XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss',
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100,
    enable_categorical=True 
)


xgb_model.fit(train_df, y_train)


y_test_pred = xgb_model.predict(test_df)
accuracy = accuracy_score(y_test, y_test_pred)
logloss = log_loss(y_test, xgb_model.predict_proba(test_df))

print(f"Accuracy: {accuracy:.2f}")
print(f"Log Loss: {logloss:.2f}")

Accuracy: 0.17
Log Loss: 2.51
